In [1]:
import pandas as pd
import re
import json
import csv

from gpt_prompt import GPTClient

In [2]:
def extract_gpt_snippets(response_text):
    # Extract the result section
    result_section = re.search(r'<result>(.*)</result>', response_text, re.DOTALL).group(1)
    # Split the result section into individual code snippets
    code_snippets = result_section.strip().split('--\n')

    return code_snippets

def prompt_user_feedback(snippets):
    """
    Prompts a user via the standard input to ask if he is satisfied with the generated snippets.
    
    If the user types in "y", "yes" or just presses enter, 
        the function returns (True, [], "") tuple
    If instead the user types in the comma-separated list of digits like "1,2,4-6,9", and specific issue to be corrected (optionally),
        the function returns (False, [1, 2, 4, 5, 6, 9], "The code snippets are too similar to each other") tuple
    """
    # Step 1: Print each snippet with a number and separator
    for i, snippet in enumerate(snippets, start=1):
        print(f"{i}.\n{snippet.strip()}")
        print('-' * 20)  # Separator for better readability

    # Step 2: Prompt user for feedback
    user_input = input("Are you satisfied with the generated snippets? (y/yes/n/no): ").strip().lower()

    # Step 3: Check user input for satisfaction
    if user_input in ['y', 'yes']:
        return True, [], ""

    # Step 4: Check if user wants to regenerate any of the samples
    user_input = input("Do you want to regenerate any of the snippets? If not, type n/no or hit Enter, " + \
                       "otherwise specify a comma-separated list of values (e.g. 1,2,4-6,9)").strip().lower()
    if user_input in ['', 'n', 'no']:
        return False, [], ""
        
    try:
        numbers = []
        for part in user_input.split(','):
            if '-' in part:
                start, end = map(int, part.split('-'))
                numbers.extend(range(start, end + 1))
            else:
                numbers.append(int(part))
        
        # Validate numbers as indices of the snippets list
        valid_indices = range(1, len(snippets) + 1)
        invalid_numbers = [num for num in numbers if num not in valid_indices]

        if invalid_numbers:
            print(f"Invalid snippet numbers: {invalid_numbers}. Please provide valid indices.")
            return prompt_user_feedback(snippets)  # Retry prompting if input is invalid

        # If the indices are correct, finally ask the user for specifying an issue with the snippets they marked incorrect
        issue = input("Please specify is there's specific issue you want to be fixed in the inputs you selected. Otherwise, press enter.").strip()
        return False, sorted(set(numbers)), issue  # Return unique sorted list of numbers
    except ValueError:
        print("Invalid input format. Please provide either 'y', 'yes', or a comma-separated list of numbers.")
        return prompt_user_feedback(snippets)  # Retry prompting if input is invalid

In [3]:
params = {
    "model": 'gpt-4o-2024-05-13',	# 'gpt-4-turbo-2024-04-09',
    "temperature": 1,
    "verbose": True
}

In [4]:
# CSV output file path
output_fname = 'result_gpt.csv'
fname_wrong  = 'dataset_wrong.csv'
fname_correct  = 'dataset_correct.csv'

df_wrong = pd.read_csv(fname_wrong, index_col='style')
df_correct = pd.read_csv(fname_correct, index_col='style')

df_wrong

,code,label,processed
style,,,
Aligned with opening delimiter.,"foo = long_function_name(var_one, var_two,\r\n...",wrong,True
Add 4 spaces (an extra level of indentation) to distinguish arguments from the rest.,"def long_function_name(\r\n var_one, va...",wrong,True
Hanging indents should add a level.,"foo = long_function_name(\r\nvar_one, var_two,...",wrong,True
No extra indentation (same level of indent).,if (this_is_one_thing and\r\n that_is_another_...,wrong,True
Add some extra indentation on the conditional continuation line.,if (this_is_one_thing\r\n and that_is_anot...,wrong,True
The closing brace/bracket/parenthesis on multiline constructs must be lined up under the first character of the line that starts the multiline construct (collection case),"my_list = [\r\n 1, 2, 3,\r\n 4, 5, 6,\r\n]",wrong,True
The closing brace/bracket/parenthesis on multiline constructs must be lined up under the first character of the line that starts the multiline construct (function call case),result = some_function_that_takes_arguments(\r...,wrong,True
"Backslashes may still be appropriate at times. For example, long, multiple with-statements could not use implicit continuation before Python 3.10, so backslashes were acceptable for that casewith open('/path/to/some/file/you/want/to/read') as file_1, \",with open('/path/to/some/file/you/want/to/read...,wrong,True
Displayed formulas always break before binary operations. Operators should be adjacent to their operands.,income = (gross_wages +\r\n taxable_i...,wrong,True


In [5]:
# Discard all the processed examples
unprocessed_df = df_wrong[~df_wrong['processed']]
unprocessed_df

,code,label,processed
style,,,
However it does not make sense to have a trailing comma on the same line as the closing delimiter (except in the case of singleton tuples):,"FILES = ['setup.cfg', 'tox.ini',]\r\ninitializ...",wrong,False
"Be consistent in return statements. Either all return statements in a function should return an expression, or none of them should. If any return statement returns an expression, any return statements where no value is returned should explicitly state this as return None, and an explicit return statement should be present at the end of the function (if reachable):",def foo(x):\r\n if x >= 0:\r\n retur...,wrong,False


In [6]:
df = unprocessed_df.join(df_correct, how='left', rsuffix='_correct')
df = df[['code', 'code_correct', 'aspects', 'label', 'processed']]

N_items = len(df.index)
N_before_join = len(unprocessed_df.index)

if N_before_join - N_items != 0:
    print(f'Warning: {N_before_join - N_items} items discarded because they didn\'t have corresponding correct snippets.')

df.reset_index(inplace=True)
df

,style,code,code_correct,aspects,label,processed
0,However it does not make sense to have a trail...,"FILES = ['setup.cfg', 'tox.ini',]\r\ninitializ...","FILES = [\r\n 'setup.cfg',\r\n 'tox.ini'...",n/cl,wrong,False
1,Be consistent in return statements. Either all...,def foo(x):\r\n if x >= 0:\r\n retur...,def foo(x):\r\n if x >= 0:\r\n retur...,n/fs/exp,wrong,False


In [7]:
def save_snippets(style, snippets, label, processed_snippets_ids):
    with open(output_fname, 'a', newline='', encoding='utf-8') as csvfile:
        csv_writer = csv.writer(csvfile, quoting=csv.QUOTE_ALL)

        for i, snippet in enumerate(snippets):
            is_processed = i+1 in processed_snippets_ids
            csv_writer.writerow([style, snippet, label, is_processed])

In [8]:
N_TO_PROCESS = 10
df = df.head(N_TO_PROCESS)

In [9]:
client = GPTClient()

for i, sample in df.iterrows():
    print(f'Sample #{i}: {sample.style}', end='\n' * 2)
    
    # Parse the aspects as the list of abbreviations that generate_correct_samples() function expects
    aspects = sample.aspects.split('/')

    # Prompt GPT and get its generated snippets based on the example from the sample row in the dataset.csv dataframe
    response = client.generate_wrong_samples(prompt_rule=sample.style, aspects_list=aspects,
                                             wrong_code=sample.code, correct_code=sample.code_correct, **params)
    response_text = response.choices[0].message.content
    if params['verbose']:
        print(f'\nASSISTANT:\n', response_text)
    
    code_snippets = extract_gpt_snippets(response_text)
    
    # Prompt user if he is satisfied with the generated snippets
    processed_answer, snippets_ids_to_regenerate, code_issue = prompt_user_feedback(code_snippets)
    last_snippets = None
    
    # If not, ask GPT to regenerate wrong snippets
    while(snippets_ids_to_regenerate):
        snippets_to_regenerate = [code_snippets[i-1] for i in snippets_ids_to_regenerate]
        
        new_snippets_response = client.regenerate_wrong_samples(snippets_to_regenerate, issue=code_issue, **params)
        new_snippets = extract_gpt_snippets(new_snippets_response.choices[0].message.content)

        for i, new_snippet in zip(snippets_ids_to_regenerate, new_snippets):
            code_snippets[i-1] = new_snippet

        last_snippets = snippets_ids_to_regenerate
        processed_answer, snippets_ids_to_regenerate, code_issue  = prompt_user_feedback(code_snippets)

    # mark last_snippets as unprocessed if the user is still dissatisfied with them:
    if processed_answer == False and last_snippets is not None:
        processed_snippets_ids = [i for i in range(1, 10 + 1) if i not in last_snippets]
    else:
        processed_snippets_ids = [i for i in range(1, 10 + 1)]
    save_snippets(sample.style, code_snippets, sample.label, processed_snippets_ids)
    
    df_wrong.loc[sample.style, 'processed'] = processed_answer
    client.clear_history()

Sample #0: However it does not make sense to have a trailing comma on the same line as the closing delimiter (except in the case of singleton tuples):


USER:
 
You are an expert programmer with extensive experience and vast knowledge of different industries. You will help me create a dataset for training an ML model to correct a code style according to the PEP 8 Python guideline.

You will be given an example of a stylistically *wrong* snippet, and your task is to generate a total of 10 similar snippets. When generating snippets, you must strictly adhere to the following instructions:

<instructions>
1. Snippets are valid pieces of Python code that must be similar to the provided <wrong_snippet> example in a sense that they *must violate* a certain <style_rule> (all relevant data are provided to you below in the corresponding XML sections). Provided example may contain multiple independent snippets, but your task is to generate one snippet at a time.
2. Snippets will be part of a ML d

Are you satisfied with the generated snippets? (y/yes/n/no):  Make your snippets contain EITHER collection definition (list, tuple) OR a function call, so that you have 5 total snippets with collection definition and 5 with function calls. Use the same snippets you already wrote (with trailing commas), but discard unnecessary code.
Do you want to regenerate any of the snippets? If not, type n/no or hit Enter, otherwise specify a comma-separated list of values (e.g. 1,2,4-6,9) 1-10


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10] range(1, 11)


Please specify is there's specific issue you want to be fixed in the inputs you selected. Otherwise, press enter. Make your snippets contain EITHER collection definition (list, tuple) OR a function call, so that you have 5 total snippets with collection definition and 5 with function calls. Use the same snippets you already wrote (with trailing commas), but discard unnecessary code.



USER:
 
There's a problem with the following snippets you generated:
<wrong_snippets>
users = ['john', 'jane',]
send_welcome_email(users, send_copy=True,)

--
items = ('apple', 'banana',)
printPurchase(items, final_check=True,)

--
config = {
    'version': '1.0',
    'timeout': 100,
    }
setup(config, apply_settings=False,)

--
ALERTS = [
    'warning',
    'error',
    ]
processAlerts(ALERTS, critical=False,)

--
info = {'name': 'Alice', 'age': 30,}
displayInfo(info, verbose=True,)

--
players = ('player1', 'player2',)
connectPlayers(players, ready=True,)

--
data = {
    'key1': 'value1',
    'key2': 'value2',
    }
processData(data, validate=True,)

--
entries = ['entry1', 
           'entry2', ]
saveEntries(entries, log=True,)

--
devices = (
    'camera', 
    'sensor', )
initializeDevices(devices, auto_connect=False,)

--
options = {
    'debug': True,
    'logs': False, }
configure(options, save=True,)
</wrong_snippets>

Now you must correct these snippets, ensuring that all 

...

Are you satisfied with the generated snippets? (y/yes/n/no):  n
Do you want to regenerate any of the snippets? If not, type n/no or hit Enter, otherwise specify a comma-separated list of values (e.g. 1,2,4-6,9) 5


[5] range(1, 11)


Please specify is there's specific issue you want to be fixed in the inputs you selected. Otherwise, press enter. 



USER:
 
There's a problem with the following snippets you generated:
<wrong_snippets>
def evaluate_num(value, threshold):
    if value > threshold:
        return value * 2
    if value < 0:
        return None

</wrong_snippets>

Now you must correct these snippets, ensuring that all the requirements are met:
1. Snippets absolutely have to violate the <style_rule> you were provided with, implementing one of the violations from <violate_instructions>.
2. Snippets have to be different from the provided examples in all relevant <other_aspects>.

Most importantly, you absolutely have to make sure to fix the following issue (ignore this if there's no specific issue specified below):


Use the same format to output the corrected snippets (only snippets of code, without comments):
<result>
{corrected_snippet_1}
--
...
--
{corrected_snippet_1}
</result>

1.
def compute_area(radius, pi_value):
    if radius < 0:
        return None
    if radius == 0:
        return
    return pi_value * radi

Are you satisfied with the generated snippets? (y/yes/n/no):  y


In [10]:
df_wrong.to_csv(fname_wrong, mode='w', index=True, quoting=csv.QUOTE_ALL)